In [3]:
import psycopg2 as db
import hashlib

def get_connection():
    conn = db.connect(
        host="localhost",
        database="cloudbox",
        user="postgres",
        password="khanzode"
    )
    print("Connection established")
    return conn

conn = get_connection()
cur = conn.cursor()

Connection established


In [ ]:
def is_uploaded(file):
    content = file.read()
    file_hash = hashlib.sha256(content)